# defining a database

Here we aim at defining a database of faces which are looking either at the left, to the centre or to the right of the screen. This will be used for further supervised learning.

For a full definition, see ``2018-03-20_LeCheapEyeTracker_build_database``


In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
np.set_printoptions(precision=2, suppress=True)
# define plots to be inserted interactively
import cv2

## a minimalistic stimulation within the notebook


Instead of using an external program to show stimuli, we will use a minimalist solution within the notebook. This will consist of chains of characters that we really display into segments:

In [2]:
targets = {}
s_target = '🔴'
s_fixat = '🔵'
s_distra = '🌫'
s_blink = '☠️'

N_dis = 36 # <<<<<<<<<<<< change to adapt to your notebook's width
N_margin = 2

targets['left'] = N_margin * s_distra + s_target + 2 * N_dis * s_distra + N_margin * s_distra

targets['blink'] = N_margin * s_distra + N_dis * s_distra + s_blink + N_dis * s_distra + N_margin * s_distra

targets['center'] = N_margin * s_distra + N_dis * s_distra + s_target + N_dis * s_distra + N_margin * s_distra

targets['right'] = N_margin * s_distra + 2 * N_dis * s_distra + s_target + N_margin * s_distra

print ('Empty line')
null = N_margin * s_distra +  (2 * N_dis + 1) * s_distra + N_margin * s_distra
print (null)
print ('Fixation dot')
center = N_margin * s_distra + N_dis * s_distra + s_fixat + N_dis * s_distra + N_margin * s_distra
print (center)

for label in targets.keys():
    print('label=', label)
    print (targets[label])

Empty line
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
Fixation dot
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🔵🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
label= left
🌫🌫🔴🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
label= blink
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫☠️🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
label= center
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🔴🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
label= right
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🔴🌫🌫


## wrapping things up

All the protocol in one script:

In [3]:
import os
datapath = 'dataset'
try:
    os.mkdir(datapath)
except:
    pass
for target in targets.keys():
    print(os.path.join(datapath, target))
    try:
        os.mkdir(os.path.join(datapath, target))
    except:
        pass

import imageio
from LeCheapEyeTracker.EyeTrackerServer import Server
from IPython.display import clear_output

N_category = len(targets.keys())
def get_stack(datapath=datapath, subject_name='anonymous', N_frame=256, startup_time=1., interframe_time=.1, waiting_time=1.2):
    try:
        import time
        time.sleep(startup_time)
        et = Server()
        print ('Time', et.clock())
        
        bootstraped, max_trials, no_trials = False, 100, 0
        while not bootstraped:
            try:
                img0 = et.cam.grab()
                #img_face, res, t0 = et.process_frame(img0, et.clock())
                bootstraped = True
            except:
                if no_trials>max_trials:
                    raise('Failed!')
                else:
                    pass
            no_trials += 1

        #H, W, three = img_face.shape
        for i in range(N_frame):            
            clear_output()
            try:
                # presentation of stimulus
                i_choice = np.random.randint(N_category)
                label = list(targets.keys())[i_choice]
                print(null)
                print(null)
                print(targets[label])
                print(null)
                print(null)

                time.sleep(waiting_time)
                frame = et.cam.grab()
                #img_face, res, t0 = et.process_frame(frame, et.clock())

                clear_output()
                print(null)
                print(null)
                print(center)
                print(null)
                print(null)

                time.sleep(interframe_time)

                filename = os.path.join(datapath, label, subject_name + '_' + '%.3d' % i + '.png')
                imageio.imwrite(filename, frame[:, :, ::-1]) # converting on the fly from BGR > RGB
            except Exception as e:
                print('Failed for trial ', i, 'with message', e)
    finally:
        et.close()

import datetime
date = datetime.datetime.now().date().isoformat()

        
get_stack(subject_name=date + '_Laurent')

🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🔵🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫🌫
Capture released


In [4]:
!ls -lR {datapath}

total 0
drwxr-xr-x  266 laurentperrinet  staff  8512 Sep 14 09:59 blink
drwxr-xr-x  293 laurentperrinet  staff  9376 Sep 14 09:59 center
drwxr-xr-x  299 laurentperrinet  staff  9568 Sep 14 09:59 left
drwxr-xr-x  293 laurentperrinet  staff  9376 Sep 14 09:59 right

dataset/blink:
total 197224
-rw-r--r--  1 laurentperrinet  staff  384921 Sep  4 22:37 2018-09-04_Laurent_100.png
-rw-r--r--  1 laurentperrinet  staff  384506 Sep  4 22:38 2018-09-04_Laurent_108.png
-rw-r--r--  1 laurentperrinet  staff  384283 Sep  4 22:38 2018-09-04_Laurent_116.png
-rw-r--r--  1 laurentperrinet  staff  385193 Sep  4 22:35 2018-09-04_Laurent_12.png
-rw-r--r--  1 laurentperrinet  staff  384489 Sep  4 22:38 2018-09-04_Laurent_121.png
-rw-r--r--  1 laurentperrinet  staff  384389 Sep  4 22:38 2018-09-04_Laurent_122.png
-rw-r--r--  1 laurentperrinet  staff  384749 Sep  4 22:39 2018-09-04_Laurent_133.png
-rw-r--r--  1 laurentperrinet  staff  384968 Sep  4 22:39 2018-09-04_Laurent_137.png
-rw-r--r--  1 laurentperrine

-rw-r--r--  1 laurentperrinet  staff  385891 Sep  4 22:35 2018-09-04_Laurent_11.png
-rw-r--r--  1 laurentperrinet  staff  385504 Sep  4 22:38 2018-09-04_Laurent_117.png
-rw-r--r--  1 laurentperrinet  staff  385392 Sep  4 22:38 2018-09-04_Laurent_120.png
-rw-r--r--  1 laurentperrinet  staff  384848 Sep  4 22:38 2018-09-04_Laurent_123.png
-rw-r--r--  1 laurentperrinet  staff  384694 Sep  4 22:38 2018-09-04_Laurent_124.png
-rw-r--r--  1 laurentperrinet  staff  385403 Sep  4 22:38 2018-09-04_Laurent_126.png
-rw-r--r--  1 laurentperrinet  staff  385577 Sep  4 22:38 2018-09-04_Laurent_131.png
-rw-r--r--  1 laurentperrinet  staff  384888 Sep  4 22:39 2018-09-04_Laurent_132.png
-rw-r--r--  1 laurentperrinet  staff  385427 Sep  4 22:39 2018-09-04_Laurent_135.png
-rw-r--r--  1 laurentperrinet  staff  384760 Sep  4 22:39 2018-09-04_Laurent_136.png
-rw-r--r--  1 laurentperrinet  staff  385341 Sep  4 22:35 2018-09-04_Laurent_14.png
-rw-r--r--  1 laurentperrinet  staff  385353 Sep  4 22:39 2018-09-0